# Chain

The basic building block of LangChain is the chain, which usually combines a Large Language Model (LLM) with a prompt, allowing for a sequence of operations on text or other data.

Let’s load our model, in this case we’ll be using OpenAI GPT model. For security reasons, we’ll load the OpenAI API key using python-dotenv package.

Note: You can signup at OpenAI for a trial version using your email and verification using your mobile number. You will have access to $5 credit for 3 months that you can use.

In [ ]:
!pip install langchain openai python-dotenv

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
OpenAIModel = "gpt-4"
llm = ChatOpenAI(model=OpenAIModel, temperature=0.1)

There are various types of chains that we can use, let’s take a look at the most common ones:

# LLMChain

LLM chain offers a combination of the LLM and a prompt, allowing sequential operations.

For instance, the LLM chain can assist in generating company names based on product descriptions.

In [ ]:
from langchain.chains import LLMChain

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?, please give 5 name in unorder list"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "The Best Queen Size Sheet Set"
chain.run(product)

In [ ]:
'1. "Supreme Slumber Sheets"\n2. "Queen Comfort Creations"\n3. "Luxury Linen Loft"\n4. "Elite Elegance Beddings"\n5. "Dreamy Duvet Designs"'

In [ ]:
product = "A good GPS product is one that provides accurate, fast, and stable navigation, along with additional features that assist users in various situations."
chain.run(product)

In [ ]:
'1. PrecisionPath Navigation Solutions\n2. AccuRoute GPS Technologies\n3. SwiftTrack Navigation Systems\n4. StableJourney GPS Innovations\n5. AssistNav GPS Solutions'

# SimpleSequentialChain

Sequential chains are composed of multiple steps or chains that can be run one after another.

The output of each step is passed as input to the next step.

The output of the last step is returned as the output of the chain.

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, verbose=True)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)


# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt, verbose=True)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=False)

In [ ]:
product = "The Best Queen Size Sheet Set"
overall_simple_chain.run(product)

In [ ]:


> Entering new LLMChain chain...
Prompt after formatting:
Human: What is the best name to describe     a company that makes The Best Queen Size Sheet Set?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a 20 words description for the following     company:"Supreme Queen Sheet Set Creations"

> Finished chain.

In [ ]:
'"Supreme Queen Sheet Set Creations specializes in crafting luxurious, high-quality queen-sized sheet sets for ultimate comfort and style."'

In [ ]:
product = "A good GPS product is one that provides accurate, fast, and stable navigation, along with additional features that assist users in various situations."
overall_simple_chain.run(product)

In [ ]:


> Entering new LLMChain chain...
Prompt after formatting:
Human: What is the best name to describe     a company that makes A good GPS product is one that provides accurate, fast, and stable navigation, along with additional features that assist users in various situations.?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a 20 words description for the following     company:"Advanced Navigation Solutions"

> Finished chain.

In [ ]:
'"Advanced Navigation Solutions provides innovative, high-tech navigation systems and solutions for various industries worldwide."'

# SequentialChain

Sequential chains are useful for complex tasks, such as translating reviews, summarizing them, detecting their language, and creating follow-up responses.

The following example shows how to create a sequential chain that translates a review, summarizes it, and then gives a follow up message in the same language as the review.

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)


# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review", verbose=True)

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary", verbose=True)

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language", verbose=True)

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up responsefrom the seller to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message", verbose=True)

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
sequentialChain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review1 = "Rendang dalam kemasan frozen yang saya coba baru-baru ini sungguh lezat dan praktis. Rasa dan aroma bumbu rendangnya sangat terasa dan dagingnya empuk, seperti rendang buatan rumah yang baru dimasak. Selain itu, kemasan frozen membuatnya sangat mudah dimasak, cukup dengan memanaskannya dalam microwave dan rendang siap saji sudah siap disajikan. Ini sangat menghemat waktu bagi saya yang memiliki jadwal yang sibuk. Pilihan rendang dalam kemasan frozen ini benar-benar menjadi pilihan tepat untuk menikmati rendang autentik kapan saja dan di mana saja. Saya sangat merekomendasikannya!"

sequentialChain(review1)

In [ ]:


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review to english:

Rendang dalam kemasan frozen yang saya coba baru-baru ini sungguh lezat dan praktis. Rasa dan aroma bumbu rendangnya sangat terasa dan dagingnya empuk, seperti rendang buatan rumah yang baru dimasak. Selain itu, kemasan frozen membuatnya sangat mudah dimasak, cukup dengan memanaskannya dalam microwave dan rendang siap saji sudah siap disajikan. Ini sangat menghemat waktu bagi saya yang memiliki jadwal yang sibuk. Pilihan rendang dalam kemasan frozen ini benar-benar menjadi pilihan tepat untuk menikmati rendang autentik kapan saja dan di mana saja. Saya sangat merekomendasikannya!

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Can you summarize the following review in 1 sentence:

The frozen packaged rendang that I recently tried is truly delicious and practical. The taste and aroma of the rendang seasoning is very noticeable and the meat is tender, like freshly cooked homemade rendang. In addition, the frozen packaging makes it very easy to cook, just by heating it in the microwave and the ready-to-serve rendang is ready to be served. This really saves time for me who has a busy schedule. Choosing this frozen packaged rendang is truly the right choice to enjoy authentic rendang anytime and anywhere. I highly recommend it!

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: What language is the following review:

Rendang dalam kemasan frozen yang saya coba baru-baru ini sungguh lezat dan praktis. Rasa dan aroma bumbu rendangnya sangat terasa dan dagingnya empuk, seperti rendang buatan rumah yang baru dimasak. Selain itu, kemasan frozen membuatnya sangat mudah dimasak, cukup dengan memanaskannya dalam microwave dan rendang siap saji sudah siap disajikan. Ini sangat menghemat waktu bagi saya yang memiliki jadwal yang sibuk. Pilihan rendang dalam kemasan frozen ini benar-benar menjadi pilihan tepat untuk menikmati rendang autentik kapan saja dan di mana saja. Saya sangat merekomendasikannya!

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a follow up responsefrom the seller to the following summary in the specified language:

Summary: The reviewer highly recommends the frozen packaged rendang for its delicious taste, tender meat, practicality, and convenience for those with a busy schedule.

Language: The language of the review is Indonesian.

> Finished chain.

> Finished chain.

In [ ]:
{'Review': 'Rendang dalam kemasan frozen yang saya coba baru-baru ini sungguh lezat dan praktis. Rasa dan aroma bumbu rendangnya sangat terasa dan dagingnya empuk, seperti rendang buatan rumah yang baru dimasak. Selain itu, kemasan frozen membuatnya sangat mudah dimasak, cukup dengan memanaskannya dalam microwave dan rendang siap saji sudah siap disajikan. Ini sangat menghemat waktu bagi saya yang memiliki jadwal yang sibuk. Pilihan rendang dalam kemasan frozen ini benar-benar menjadi pilihan tepat untuk menikmati rendang autentik kapan saja dan di mana saja. Saya sangat merekomendasikannya!',
 'English_Review': 'The frozen packaged rendang that I recently tried is truly delicious and practical. The taste and aroma of the rendang seasoning is very noticeable and the meat is tender, like freshly cooked homemade rendang. In addition, the frozen packaging makes it very easy to cook, just by heating it in the microwave and the ready-to-serve rendang is ready to be served. This really saves time for me who has a busy schedule. Choosing this frozen packaged rendang is truly the right choice to enjoy authentic rendang anytime and anywhere. I highly recommend it!',
 'summary': 'The reviewer highly recommends the frozen packaged rendang for its delicious taste, tender meat, practicality, and convenience for those with a busy schedule.',
 'followup_message': 'Terima kasih banyak atas ulasan positif Anda! Kami sangat senang mengetahui bahwa Anda menikmati rasa rendang beku kami dan menemukan praktis untuk jadwal Anda yang sibuk. Kami berkomitmen untuk selalu menyediakan makanan berkualitas tinggi yang lezat dan mudah disiapkan. Terima kasih telah merekomendasikan produk kami. Selamat menikmati!'}

In [ ]:
review2 = "Saya baru saja mencoba rendang dalam kemasan frozen dan sayangnya, pengalaman saya kurang memuaskan. Meski kemasannya praktis dan mudah dimasak, rasa rendangnya sendiri tidak memenuhi ekspektasi. Bumbu rendangnya kurang terasa dan dagingnya agak keras, tidak seperti rendang buatan rumah yang saya harapkan. Selain itu, saya merasa rendang ini terlalu berminyak yang membuat saya merasa tidak nyaman setelah memakannya. Sayang sekali, saya mungkin tidak akan membeli produk ini lagi di masa mendatang, kecuali ada perbaikan pada rasa dan kualitas dagingnya."

sequentialChain(review2)

In [ ]:


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review to english:

Saya baru saja mencoba rendang dalam kemasan frozen dan sayangnya, pengalaman saya kurang memuaskan. Meski kemasannya praktis dan mudah dimasak, rasa rendangnya sendiri tidak memenuhi ekspektasi. Bumbu rendangnya kurang terasa dan dagingnya agak keras, tidak seperti rendang buatan rumah yang saya harapkan. Selain itu, saya merasa rendang ini terlalu berminyak yang membuat saya merasa tidak nyaman setelah memakannya. Sayang sekali, saya mungkin tidak akan membeli produk ini lagi di masa mendatang, kecuali ada perbaikan pada rasa dan kualitas dagingnya.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Can you summarize the following review in 1 sentence:

I just tried the frozen packaged rendang and unfortunately, my experience was less than satisfactory. Although the packaging is practical and easy to cook, the taste of the rendang itself did not meet my expectations. The rendang seasoning was not flavorful enough and the meat was somewhat tough, unlike the homemade rendang I was hoping for. In addition, I felt this rendang was too oily, which made me feel uncomfortable after eating it. It's a shame, I probably won't buy this product again in the future, unless there are improvements in its taste and meat quality.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: What language is the following review:

Saya baru saja mencoba rendang dalam kemasan frozen dan sayangnya, pengalaman saya kurang memuaskan. Meski kemasannya praktis dan mudah dimasak, rasa rendangnya sendiri tidak memenuhi ekspektasi. Bumbu rendangnya kurang terasa dan dagingnya agak keras, tidak seperti rendang buatan rumah yang saya harapkan. Selain itu, saya merasa rendang ini terlalu berminyak yang membuat saya merasa tidak nyaman setelah memakannya. Sayang sekali, saya mungkin tidak akan membeli produk ini lagi di masa mendatang, kecuali ada perbaikan pada rasa dan kualitas dagingnya.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a follow up responsefrom the seller to the following summary in the specified language:

Summary: The reviewer found the frozen rendang disappointing due to its lack of flavor, tough meat, and excessive oiliness, and won't repurchase unless the taste and meat quality improve.

Language: The language of the review is Indonesian.

> Finished chain.

> Finished chain.

In [ ]:
{'Review': 'Saya baru saja mencoba rendang dalam kemasan frozen dan sayangnya, pengalaman saya kurang memuaskan. Meski kemasannya praktis dan mudah dimasak, rasa rendangnya sendiri tidak memenuhi ekspektasi. Bumbu rendangnya kurang terasa dan dagingnya agak keras, tidak seperti rendang buatan rumah yang saya harapkan. Selain itu, saya merasa rendang ini terlalu berminyak yang membuat saya merasa tidak nyaman setelah memakannya. Sayang sekali, saya mungkin tidak akan membeli produk ini lagi di masa mendatang, kecuali ada perbaikan pada rasa dan kualitas dagingnya.',
 'English_Review': "I just tried the frozen packaged rendang and unfortunately, my experience was less than satisfactory. Although the packaging is practical and easy to cook, the taste of the rendang itself did not meet my expectations. The rendang seasoning was not flavorful enough and the meat was somewhat tough, unlike the homemade rendang I was hoping for. In addition, I felt this rendang was too oily, which made me feel uncomfortable after eating it. It's a shame, I probably won't buy this product again in the future, unless there are improvements in its taste and meat quality.",
 'summary': "The reviewer found the frozen rendang disappointing due to its lack of flavor, tough meat, and excessive oiliness, and won't repurchase unless the taste and meat quality improve.",
 'followup_message': 'Terima kasih atas ulasan Anda. Kami sangat menyesal mendengar bahwa Anda kecewa dengan rendang beku kami. Kami sangat menghargai masukan Anda mengenai rasa dan kualitas daging, dan kami akan membawanya ke tim produksi kami untuk perbaikan. Kami berharap Anda akan memberi kami kesempatan lain untuk memperbaiki pengalaman Anda. Terima kasih telah memilih produk kami.'}

# Router Chain

Router chain is a special type of chain, the multi-prompt chain, can route an input to a specific chain depending on the input type.

The router chain is a chain that has a list of chains and a list of prompts. The router chain will prompt the user with the list of prompts and then route the input to the chain that corresponds to the prompt.

For example, we will create a router chain that will respond to questions based on the subject of the question.

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
# Prompt templates for the chain of reasoning physics question

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

In [ ]:
# Prompt template for chain of reasoning math question

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

In [ ]:
# Prompt template for chain of reasoning history question

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

In [ ]:
# Prompt template for chain of reasoning computer science question

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

For instance, four chains can be created for different subjects: physics, math, history, and computer science. Depending on the input question, the router chain directs it to the appropriate chain.

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

If none of the specific chains can accommodate the question, it gets directed to a default chain.

The default chain is a catch-all chain that can answer any question.

The default chain is the last chain in the list of chains.

The default chain is optional. If you do not specify a default chain, the question is not answered.

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt, verbose=True)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
routerChain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
routerChain.run("What is black body radiation?")

In [ ]:


> Entering new MultiPromptChain chain...

In [ ]:
/home/eddypermana22/.local/lib/python3.8/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(

In [ ]:
physics: {'input': 'What is black body radiation?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.

Here is a question:
What is black body radiation?

> Finished chain.

> Finished chain.

In [ ]:
"Black body radiation is the type of electromagnetic radiation that is emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. The radiation has a specific spectrum and intensity that depends only on the body's temperature, which is assumed to be uniform and constant. This radiation includes visible light, infrared light, ultraviolet light, and so on. The theory of black body radiation is fundamental to the development of quantum mechanics."

In [ ]:
routerChain.run("what is 2 + 2")

In [ ]:


> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them togetherto answer the broader question.

Here is a question:
what is 2 + 2

> Finished chain.

> Finished chain.

In [ ]:
'The answer to 2 + 2 is 4.'

In [ ]:
routerChain.run("Why does every cell in our body contain DNA?")

In [ ]:


> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Why does every cell in our body contain DNA?

> Finished chain.

> Finished chain.

In [ ]:
"DNA, or deoxyribonucleic acid, is the hereditary material in humans and almost all other organisms. It contains the instructions needed for an organism to develop, survive and reproduce. These instructions are found inside every cell, and are passed down from parents to their children.\n\nDNA is organized into structures called chromosomes and is divided into coding regions called genes. Each gene contains the instructions for the production of a specific protein. Proteins are complex molecules that play many critical roles in the body. They are required for the structure, function, and regulation of the body's tissues and organs.\n\nTherefore, every cell in our body contains DNA because it is needed for the cell's structure, function, and reproduction. Without DNA, our cells would not know how to function, and could not pass on our genes to the next generation."

In [ ]:
routerChain.run("How did the Dutch colonization influence the culture and economy of Indonesia?")

In [ ]:


> Entering new MultiPromptChain chain...
History: {'input': 'How did the Dutch colonization influence the culture and economy of Indonesia?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very good historian. You have an excellent knowledge of and understanding of people,events and contexts from a range of historical periods. You have the ability to think, reflect, debate, discuss and evaluate the past. You have a respect for historical evidenceand the ability to make use of it to support your explanations and judgements.

Here is a question:
How did the Dutch colonization influence the culture and economy of Indonesia?

In [ ]:
routerChain.run("What is the concept of 'Big Data', and how has it impacted the field of Computer Science? Can you give an example of its application in real life?")